In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
IMG_SIZE = 64  # 画像のサイズ

In [ ]:

# ====== パスの設定 ======
model_path = './ml'
test_path = './ml/test'
model_file = os.path.join(model_path, 'model_50epochs.h5')
label_file = os.path.join(model_path, 'class_indices.json')
test_root = './kagglehub_cache/datasets/gpiosenka/sports-classification/versions/9/test'

# ====== モデルとラベルの読み込み ======
model = load_model(model_file)

with open(label_file, 'r', encoding='utf-8') as f:
    class_indices = json.load(f)
class_labels = {int(v): k for k, v in class_indices.items()}

# ====== 推論処理 ======
results = []
image_id = 1

for true_class in sorted(os.listdir(test_root)):
    class_dir = os.path.join(test_root, true_class)
    if not os.path.isdir(class_dir):
        continue

    for fname in sorted(os.listdir(class_dir)):
        fpath = os.path.join(class_dir, fname)
        if not fpath.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        # 画像読み込みと前処理
        img = image.load_img(fpath, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

        # 予測
        preds = model.predict(img_array, verbose=0)[0]
        top_indices = preds.argsort()[-3:][::-1]
        top_labels = [class_labels[i] for i in top_indices]

        # 結果記録
        results.append([
            image_id,
            true_class,
            top_labels[0],
            top_labels[1],
            top_labels[2]
        ])
        image_id += 1

# ====== CSV保存 ======
df_results = pd.DataFrame(results, columns=["画像ID", "正解", "Top1予測", "Top2予測", "Top3予測"])
output_csv_path = os.path.join(test_path, "prediction_results.csv")
df_results.to_csv(output_csv_path, index=False, encoding='utf-8')
print(f"予測結果をCSVとして保存しました。")


In [ ]:
import pandas as pd
from collections import Counter

# CSV読み込み
df = pd.read_csv(os.path.join(test_path, "prediction_results.csv"))

# Top1・Top3 精度算出
top1_correct = df['正解'] == df['Top1予測']
top3_correct = df.apply(lambda row: row['正解'] in [row['Top1予測'], row['Top2予測'], row['Top3予測']], axis=1)

top1_accuracy = top1_correct.mean()
top3_accuracy = top3_correct.mean()

print(f"🎯 Top1精度: {top1_accuracy:.2%}")
print(f"🎯 Top3精度: {top3_accuracy:.2%}")

# クラス別のTop1正解率
class_accuracy = df.groupby('正解')['Top1予測'].apply(lambda x: (x == x.name).mean())
print("\n📈 クラス別 Top1精度（上位10クラス）:")
print(class_accuracy.sort_values(ascending=False).head(10))

# Top1で間違えた場合の相手集計
errors = df[~top1_correct]
confusions = errors.groupby(['正解', 'Top1予測']).size().reset_index(name='件数')
confusions = confusions.sort_values('件数', ascending=False)
print("\n🌀 よくある混同例（Top10）:")
print(confusions.head(10))

# 正解・Top1予測が異なる例（抜粋）
print("\n🔍 予測ミスの例（5件）:")
print(errors[['画像ID', '正解', 'Top1予測', 'Top2予測', 'Top3予測']].head(5))
